# Урок 3. Коллаборативная фильтрация

1) Попытаться ответить на вопросы/выдвинуть гипотезы

**Ситуация**: Вы работает data scientist в крупном продуктовом российском ритейлере. Ваш конкурент сделал рекомендательную систему, и его продажи выросли. Ваш менеджмент тоже хочет увеличить продажи   

**Задача со слов менеджера**: Сделайте рекомендательную систему топ-10 товаров для рассылки по e-mail

**Ожидание:**
- Отправляем e-mail с топ-10 товарами, отсортированными по вероятности

**Реальность:**
- Чего хочет менеджер от рекомендательной системы? (рост показателя X на Y% за Z недель)

увеличить прибыль
- По-хорошему надо бы предварительно посчитать потенциальный эффект от рекоммендательной системы (Оценки эффектов у менеджера и у вас могут сильно не совпадать: как правило, вы знаете про данные больше)

все зависит от изначальной цели

- А у нас вообще есть e-mail-ы пользователей? Для скольки %? Не устарели ли они?

нужно бы выделить ту часть пользователей, которых раздражают рассылки, чтобы по итогу их не потерять как клиентов.

- Будем ли использовать СМС и push-уведомления в приложении? Может, будем печатать рекомендации на чеке после оплаты на кассе?

надо провести опрос среди клиентов, как бы они педпочли получать информацию. через смс, push-уведомления или email

- Как будет выглядеть e-mail? (решаем задачу топ-10 рекомендаций или ранжирования? И топ-10 ли?)

топ10 многовато. максимум 5

- Какие товары должны быть в e-mail? Есть ли какие-то ограничения (только акции и т п)?

на которые сейчас максимальные скидки

- Сколько денег мы готовы потратить на привлечение 1 юзера? CAC - Customer Aquisition Cost. Обычно CAC = расходы на коммуникацию + расходы на скидки
- Cколько мы хотим зарабатывать с одного привлеченного юзера?
---
- А точно нужно сортировать по вероятности?

да
- Какую метрику использовать?

Precision. 
- Сколько раз в неделю отпрпавляем рассылку?
не чаще одного раза в неделю
- В какое время отправляем рассылку?
либо рано утром, либо вечером после рабочего дня, но надо учитывать еще разность в часовых поясах.

- Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?

исключить повторы одних и тех же рекомендаций. ну или показывать 1-2 с наибольшими скидками
- Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?

можно. если делать рассылку раз в неделю, то можно в одну неделю отправить товары категории "товары для дома" а в следующий раз "косметика" еще было бы неплохо приурочить рассылки к ближайшим праздникам.
- И многое другое:)

**В итоге договорились, что:**
- Хотим повысить выручку минимум на 6% за 4 месяца. Будем повышать за счет роста Retention минимум на  3% и среднего чека минимум на 3%
- Топ-5 товаров, а не топ-10 (В e-mail 10 выглядят не красиво, в push и на чек больше 5 не влезает)
- Рассылаем в e-mail (5% клиентов) и push-уведомлении (20% клиентов), печатаем на чеке (все оффлайн клиенты)
- **3 товара с акцией** (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)
- **1 новый товар** (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?) 
- **1 товар для роста среднего чека** (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)


2) Поэкспериментировать с ALS (grid-search)

In [2]:


import itertools as it

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Для поиска параметров ALS
from sklearn.model_selection import GridSearchCV

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
    
from metrics import precision_at_k, recall_at_k


%matplotlib inline



In [3]:
data = pd.read_csv('retail_train.csv')

In [4]:
data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
            inplace=True)

In [5]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [7]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

In [8]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

In [9]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [10]:
# Заведем фиктивный item_id
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

C:\Users\tatya\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [11]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [12]:
# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix)

In [13]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [14]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [15]:
def score_als_precision(model, user_ids, actual_recs, N=5):
#     model.fit(csr_matrix(user_item_matrix).T,  # На вход item-user matrix
#               show_progress=True)

#     recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
#                            user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
#                            N=5, # кол-во рекомендаций 
#                            filter_already_liked_items=False, 
#                            filter_items=None, 
#                            recalculate_user=True)
    
    recs = user_ids.apply(lambda x: get_recommendations(x, model=model, N=N))
    result = pd.DataFrame(actual_recs)
    result = result.rename(columns={'user_id': 'actual'})
    result['recs'] = recs
    score = result.apply(lambda row: precision_at_k(row['recs'], row['actual']), axis=1).mean()
    
    return score

In [16]:
rec_results = data_test.groupby('user_id')['item_id'].unique().reset_index()
rec_results.columns=['user_id', 'actual']
rec_results.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [17]:
parameters = {'factors': [32, 64, 128],
              'regularization': [0.0001, 0.001, 0.01],
              'iterations': [10, 15, 20]}

In [18]:
%%time

scores = []
for factors, reg, iters in it.product(*parameters.values()):
    model = AlternatingLeastSquares(factors=factors, 
                                    regularization=reg,
                                    iterations=iters, 
                                    calculate_training_loss=True, 
                                    num_threads=6)

    model.fit(csr_matrix(user_item_matrix).T,  # На вход item-user matrix
              show_progress=True)
    
    score = {'factors': factors,
             'regularization': reg,
             'iterations': iters,
             'score': score_als_precision(model, rec_results['user_id'], rec_results['actual'], N=5)}
    scores.append(score)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Wall time: 8min 56s


In [19]:
scores

[{'factors': 32,
  'regularization': 0.0001,
  'iterations': 10,
  'score': 0.16846229187071265},
 {'factors': 32,
  'regularization': 0.0001,
  'iterations': 15,
  'score': 0.16787463271302402},
 {'factors': 32,
  'regularization': 0.0001,
  'iterations': 20,
  'score': 0.1654260528893219},
 {'factors': 32,
  'regularization': 0.001,
  'iterations': 10,
  'score': 0.16062683643486564},
 {'factors': 32,
  'regularization': 0.001,
  'iterations': 15,
  'score': 0.1649363369245814},
 {'factors': 32,
  'regularization': 0.001,
  'iterations': 20,
  'score': 0.16611165523995844},
 {'factors': 32,
  'regularization': 0.01,
  'iterations': 10,
  'score': 0.16738491674828362},
 {'factors': 32,
  'regularization': 0.01,
  'iterations': 15,
  'score': 0.16758080313417983},
 {'factors': 32,
  'regularization': 0.01,
  'iterations': 20,
  'score': 0.16856023506366072},
 {'factors': 64,
  'regularization': 0.0001,
  'iterations': 10,
  'score': 0.16317335945151606},
 {'factors': 64,
  'regularizat

In [20]:
pd.DataFrame(scores, index=range(len(scores))).sort_values('score', ascending=False)

,factors,regularization,iterations,score
8,32,0.0100,20,0.168560
0,32,0.0001,10,0.168462
15,64,0.0100,10,0.168364
1,32,0.0001,15,0.167875
12,64,0.0010,10,0.167777
7,32,0.0100,15,0.167581
6,32,0.0100,10,0.167385
5,32,0.0010,20,0.166112
16,64,0.0100,15,0.166014
2,32,0.0001,20,0.165426
